In [1]:
import pandas as pd
import numpy as np

In [2]:
column_order = ['ID Resultado', 'ID Competidor', 'Nome Competidor',
                'ID Competição', 'Classe Vela', 'Pontuação Regata',
                'Descarte', 'Flotilha', 'Posição Geral', 'Punição',
                'Pontuação Total', 'Nett', 'Nome Competição', 'Regata']

In [3]:
df2 = pd.read_csv('../scraped-data-2024/cluster2-html.csv')
df2 = pd.concat([df2, pd.read_csv('../scraped-data-2024/cluster2_5-pdf.csv')])

In [4]:
df2.replace('\xa0', np.nan, inplace=True)
df2.dropna(subset=["Pontuação Regata"], inplace=True)
df2['Punição'] = 0
df2['Descarte'] = 0

In [5]:
import re
import unidecode
import numpy as np

def process_string(s):
    # Remove espaços em branco extras
    s = str(s).strip()
    
    numbers = re.findall(r'\d+(?:\.\d+)?', s)
    if numbers == []:
        numbers.append(np.nan)
    
    has_parentheses = 1 if '(' in s or ')' in s else 0
    
    letters = ''.join(re.findall(r'[a-zA-Z]', s))
    if letters == '':
        letters = 0
    return numbers, has_parentheses, letters

In [6]:
df2['Punição'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[2])
df2['Descarte'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[1])
df2['Pontuação Regata'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[0][0])
df2["Nome Competidor"] = df2["Nome Competidor"] .apply(lambda x: unidecode.unidecode(x).upper())

In [7]:
df2['ID Resultado'] = ''
df2['ID Competidor'] = ''

In [8]:
df2 = df2[column_order]

In [9]:
df2.to_csv('../cleaned_data_2024/cluster_2.csv', index=False)

In [10]:
df3 = pd.read_csv('../scraped-data-2024/cluster3.csv')

In [11]:
df3["Nome Competidor"] = df3["Nome Competidor"].apply(lambda x: str(x).strip().replace('/', ', '))
df3["Nome Competidor"] = df3["Nome Competidor"].apply(lambda x: unidecode.unidecode(x).upper())

In [12]:
ids = ['ID Resultado', 'ID Competidor', 'ID Competição']
for id in ids:
    df3[id] = ''

In [13]:
df3 = df3[column_order]

In [14]:
df3.to_csv('../cleaned_data_2024/cluster_3.csv', index=False)

In [15]:
df6 = pd.read_csv('../scraped-data-2024/cluster6-pdf.csv')

In [16]:
df6["Nome Competidor"] = df6["Nome Competidor"].apply(lambda x: unidecode.unidecode(x).upper())
df6['Descarte'] = 0
df6[['ID Resultado', 'ID Competidor']] = ''

In [17]:
df6 = df6[column_order]

In [18]:
df6.to_csv('../cleaned_data_2024/cluster6.csv', index=False)

In [19]:
df7 = pd.read_csv('../scraped-data-2024/cluster7-pdf.csv')

In [20]:
def clean_names_7(line):
    results = line.strip().split(' - ')
    if len(results) > 1:
        punicao = results[0].strip("[")

        if punicao == results[0]:
            descarte = 0
            val = results[1].strip()

        else:
            descarte = 1
            val = results[1].strip("]")
    else:
        punicao = 0
        val = line.strip('[').strip(']')
        if val == line:
            descarte = 0
        else:
            descarte = 1

    return descarte, punicao, val

In [21]:
df7['Descarte'] = df7['Pontuação Regata'].apply(lambda x: clean_names_7(x)[0])
df7['Punição'] = df7['Pontuação Regata'].apply(lambda x: clean_names_7(x)[1])
df7['Pontuação Regata'] = df7['Pontuação Regata'].apply(lambda x: clean_names_7(x)[2])

In [22]:
df7['Nome Competidor'] = df7['Nome Competidor'].apply(lambda x: unidecode.unidecode(x).upper())
df7[['ID Resultado', 'ID Competidor']] =  ''

In [23]:
df7 = df7[column_order]

In [24]:
df7.to_csv('../cleaned_data_2024/cluster7.csv', index=False)

In [25]:
df8 = pd.read_csv('../scraped-data-2024/cluster8-html.csv')

In [26]:
df8 = df8.dropna(subset=['Pontuação Regata'])
df8["Nome Competidor"] = df8["Nome Competidor"].apply(lambda x: unidecode.unidecode(str(x)))

In [27]:
def sep_nomes(nomes):
    nomes_list = str(nomes).strip().split(" ")
    new_nomes_list = list()

    for i in range(len(nomes_list)):
        if nomes_list[i] != nomes_list[i].title() and nomes_list[i] != nomes_list[i].upper():
            nnomee = ''
            nome = nomes_list[i]
            for i in range(len(nome)-1):
                if nome[i] == nome[i].upper() and nome[i+1] == nome[i+1].lower():
                    nnomee += ', '

                nnomee += nome[i]

            nnomee += nome[-1]
            nnomee += ' '

            new_nomes_list.append(nnomee)
        else:
            new_nomes_list.append(nomes_list[i])
            new_nomes_list.append(' ')

    new_nome = ''
    for name in new_nomes_list:
        new_nome += name
    
    return new_nome

In [28]:
df8['Nome Competidor'] = df8['Nome Competidor'].apply(lambda x: sep_nomes(x).strip())
df8['Nome Competidor'] = df8['Nome Competidor'].apply(lambda x: unidecode.unidecode(x).upper())

In [29]:
def get_descarte(line):
    descarte = 0
    if type(line) == str:
        if str(line).startswith('(') or str(line).endswith(')'):
            descarte = 1
    val = str(line).strip('(').strip(')')

    return descarte, val

In [30]:
df8['Descarte'] = df8['Pontuação Regata'].apply(lambda x: get_descarte(x)[0])
df8['Pontuação Regata'] = df8['Pontuação Regata'].apply(lambda x: get_descarte(x)[1])
df8[['ID Resultado', 'ID Competidor']] = ''

In [31]:
df8 = df8[column_order]

In [32]:
df8.to_csv('../cleaned_data_2024/cluster8.csv', index=False)

In [33]:
df9 = pd.read_csv('../scraped-data-2024/cluster9-html.csv')

In [34]:
df9 = df9.dropna(subset=['Pontuação Regata'])
df9["Nome Competidor"] = df9["Nome Competidor"].apply(lambda x: unidecode.unidecode(str(x)))
df9['Nome Competidor'] = df9['Nome Competidor'].apply(lambda x: sep_nomes(x).strip())
df9['Nome Competidor'] = df9['Nome Competidor'].apply(lambda x: unidecode.unidecode(x).upper())
df9[['ID Resultado', 'ID Competidor']] = ''

In [35]:
df9 = df9[column_order]

In [36]:
df9.to_csv('../cleaned_data_2024/cluster9.csv', index=False)

In [37]:
df14 = pd.read_csv('../scraped-data-2024/cluster14-html.csv')

In [38]:
df14 = df14.dropna(subset=['Pontuação Regata'])
df14["Nome Competidor"] = df14["Nome Competidor"].apply(lambda x: unidecode.unidecode(str(x)))

In [39]:
df14.to_csv('../cleaned_data_2024/cluster14.csv', index=False)

ID Resultado, ID Competidor, Nome Competidor, ID Competição, Classe Vela, Pontuação Regata, Descarte, Flotilha, Posição Geral, Punição, Pontuação Total, Nett, Nome Competição, Regata


In [ ]:
import os
import pandas as pd
files = [f for f in os.listdir('../cleaned_data_2024') if f.endswith('.csv')]

df_init = pd.read_csv("../cleaned_data_2024/"+files[0])
for file in files[1:]:
    df_init = pd.concat([df_init, pd.read_csv("../cleaned_data_2024/"+file)])
df_init.to_csv("../cleaned_data_2024/final_data.csv", index = False)    